In [1]:
#import pandas as pd
import numpy as np
import requests as rq

# Collect tickers form wikipedia

In [7]:
data =  {"Compamy":[]}    ### create dictionary to hold ticker symbols
wikiURL = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
wikiResponse =  rq.get(wikiURL)     ##use requests library to retreive info from webpage
tst =   wikiResponse.text     #just get the text from the requests object

text = tst.split('<tr>')
ticker=[]
for i in range(2,505):
    ticker.append(text[i].split('">')[1].split('</')[0])
data["Company"]=ticker

In [ ]:
# Collect data from Yahoo Finance site

In [8]:
%%time
Indicators = {'Previous Close':[],
              'Open':[],
              'Bid':[],
              'Ask':[],
              'PE Ratio':[],
              'Volume':[],
              "Avg. Volume":[],
              'Market Cap':[],
              'Beta':[],
              'EPS (TTM)':[]}
for tkr in data["Company"][:20]:
    url = 'https://finance.yahoo.com/quote/' + tkr
    try:
        yahooResponse = rq.get(url).text
        op = yahooResponse.split('OPEN-value">')[1]
        pvc = yahooResponse.split('PREV_CLOSE-value">')[1]
        bid = yahooResponse.split('BID-value">')[1]
        ask = yahooResponse.split('ASK-value">')[1]
        pe = yahooResponse.split('PE_RATIO-value">')[1]
        vol = yahooResponse.split('TD_VOLUME-value">')[1]
        avgVol = yahooResponse.split('AVERAGE_VOLUME_3MONTH-value">')[1]
        mktCap = yahooResponse.split('MARKET_CAP-value">')[1]
        beta = yahooResponse.split('BETA_5Y-value">')[1]
        eps = yahooResponse.split('EPS_RATIO-value">')[1]
        
        Indicators["Open"].append(op.split('</')[0])
        Indicators["Previous Close"].append(pvc.split('</')[0])
#         Indicators["Bid"].append(bid.split('</')[0].split(' x')[0])
#         Indicators["Ask"].append(ask.split('</')[0].split(' x')[0])
        Indicators["Bid"].append(bid.split('</')[0])
        Indicators["Ask"].append(ask.split('</')[0])
        Indicators["PE Ratio"].append(pe.split('</')[0])
        Indicators["Volume"].append(vol.split('</')[0].split('active="">')[1])
        Indicators["Avg. Volume"].append(avgVol.split('</')[0])
        Indicators["Market Cap"].append(mktCap.split('</')[0])
        Indicators["Beta"].append(beta.split('</')[0])
        Indicators["EPS (TTM)"].append(eps.split('</')[0])
           
    except:
        Indicators["Open"].append('NA')
        Indicators["Previous Close"].append('NA')
        print('error @' + tkr)

Wall time: 12.8 s


### Update dictionary and create pandas dataframe:

In [9]:
import pandas as pd
data.update()
data_df = pd.DataFrame.from_dict(Indicators)
data_df.head(10)

,Previous Close,Open,Bid,Ask,PE Ratio,Volume,Avg. Volume,Market Cap,Beta,EPS (TTM)
0,112.30,112.00,110.51 x 800,111.10 x 1100,15.43,"3,112,788","9,686,761",61.174B,0.99,7.16
1,48.73,48.74,48.71 x 800,48.81 x 800,14.95,"970,275","979,165",7.503B,1.20,3.25
2,98.29,98.71,97.48 x 1300,98.31 x 900,20.20,"6,728,558","4,713,676",169.448B,0.71,4.79
3,142.72,142.20,134.20 x 900,135.42 x 1000,18.98,"15,358,992","5,507,111",237.297B,0.69,7.07
4,248.96,249.53,244.77 x 900,247.40 x 900,52.27,"427,177","261,371",11.168B,1.38,4.70
5,258.27,258.71,257.12 x 1000,258.97 x 1100,24.02,"2,924,375","1,987,490",162.23B,1.22,10.71
6,74.87,74.90,74.26 x 3000,74.56 x 2200,31.10,"9,498,212","5,224,568",58.157B,0.51,2.39
7,80.61,81.09,80.59 x 1200,80.78 x 1100,12.69,"2,436,208","2,645,522",45.097B,0.77,6.34
8,278.25,279.69,276.00 x 800,276.99 x 1300,27.44,"4,578,776","3,493,993",127.94B,1.13,10.03
9,227.75,227.77,225.64 x 1000,227.35 x 800,32.31,"2,875,272","1,609,961",93.935B,0.83,7.00


#### cleanup ask and bid columns

In [10]:
data_df.Ask[10]

'157.56 x 1000'

In [11]:
data_df['Ask'] = data_df['Ask'].apply(lambda x: x.split(' x')[0])
data_df['Bid'] = data_df['Bid'].apply(lambda x: x.split(' x')[0])

In [12]:
data_df.head(12)

,Previous Close,Open,Bid,Ask,PE Ratio,Volume,Avg. Volume,Market Cap,Beta,EPS (TTM)
0,112.30,112.00,110.51,111.10,15.43,"3,112,788","9,686,761",61.174B,0.99,7.16
1,48.73,48.74,48.71,48.81,14.95,"970,275","979,165",7.503B,1.20,3.25
2,98.29,98.71,97.48,98.31,20.20,"6,728,558","4,713,676",169.448B,0.71,4.79
3,142.72,142.20,134.20,135.42,18.98,"15,358,992","5,507,111",237.297B,0.69,7.07
4,248.96,249.53,244.77,247.40,52.27,"427,177","261,371",11.168B,1.38,4.70
5,258.27,258.71,257.12,258.97,24.02,"2,924,375","1,987,490",162.23B,1.22,10.71
6,74.87,74.90,74.26,74.56,31.10,"9,498,212","5,224,568",58.157B,0.51,2.39
7,80.61,81.09,80.59,80.78,12.69,"2,436,208","2,645,522",45.097B,0.77,6.34
8,278.25,279.69,276.00,276.99,27.44,"4,578,776","3,493,993",127.94B,1.13,10.03
9,227.75,227.77,225.64,227.35,32.31,"2,875,272","1,609,961",93.935B,0.83,7.00
